In [ ]:
from pathlib import Path
import torch
import numpy as np
path = Path("/scratch/dldevel/sinziri/Evolutionary_Diffusion_Enhancement/results/a_cat_1549660/")

# Fitness Scores over Generations

In [ ]:
pil_path = path / "images"
# for i in pil_path.iterdir():
#     print(i)
glob = pil_path.glob("*.JPEG")
results =[] 
for p in glob:
    
    results.append(p.name)

print(len(results))

In [ ]:
data = [r.split("_") for r in results]
print(len(data))
print(data[0])

data_x = [{"gen" : int(d[0].replace("g", "")),
          "id" : d[2],
          "score" : float(d[-1].replace("f", "").replace(".JPEG", ""))} for d in data]

print(data_x[0])
print("test")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame(data_x)
df = df.sort_values(by='gen', ascending=True)
generations = df['gen'].unique()


grouped_data = [df['score'][df['gen'] == g].values for g in generations]
labels = generations

fig, ax = plt.subplots(figsize=(10, 6))

violins = ax.violinplot(
    grouped_data,

    positions=np.arange(1, len(labels) + 1), 
    vert=True,            
    showmeans=True,       
    showmedians=True,     
    showextrema=True      
)


for pc in violins['bodies']:
    pc.set_facecolor('#4682B4')  # Stahlblau
    pc.set_edgecolor('black')
    pc.set_alpha(0.7)


ax.set_title('Distribution of Fitness over Generations')
ax.set_ylabel('Fitness')
ax.set_xlabel('Generation')


ax.set_xticks(np.arange(1, len(labels) + 1))
ax.set_xticklabels(labels)

plt.grid(True, linestyle='--', alpha=0.5, axis='y')
plt.tight_layout()
plt.show()

# Structure Analysis of the BLIP2 Embeddings

In [ ]:
blip2_embeddings_path = path / "blip2"

glob = blip2_embeddings_path.glob("*.pt")
embds =[]
gen = []
for i in glob:
    print(f"\r{i}", flush=True, end='')
    embds.append(torch.load(i, map_location="cpu"))
    gen.append(int(str(i.name).split("_")[1].replace("g","")))
print(len(embds))
print(gen[0])

In [ ]:

print(embds[0].shape)
# flat_tensors = [t.squeeze(0) for t in embds]
stacked_tensor = torch.cat(embds, dim=0)
data_matrix = stacked_tensor.numpy()
# data_matrix = np.reshape(data_matrix, -1)
print(f"Shape of Data: {data_matrix.shape}")

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data_matrix)

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95)
principal_components = pca.fit_transform(scaled_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
gen_nums = np.array([int(g) for g in gen])
min_gen = gen_nums.min() 
max_gen = gen_nums.max() 


plt.figure(figsize=(8, 6))



plt.scatter(
    principal_components[:, 0],
    principal_components[:, 1],
    c=gen_nums,
    cmap='viridis',
    s=5,
    alpha=0.7)
plt.title('2D-PCA-Plot (PC1 vs. PC2)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True)
plt.show()

In [ ]:
blip2_explained_variance = pca.explained_variance_ratio_
print("Explained Variance: (PC1 - PC4):")
for i in range(4):
    variance_percentage = blip2_explained_variance[i] * 100
    print(f"Komponente {i+1} (PC{i+1}): {variance_percentage:.2f}%")

# Noise Analysis

In [ ]:
path_initial_noise = path / "initial_noise"

glob = path_initial_noise.glob("*.pt")
embds =[]
gen = []
for i in glob:
    print(f"\r{i}", flush=True, end='')
    embds.append(torch.load(i, map_location="cpu"))
    gen.append(int(str(i.name).split("_")[0].replace("g","")))
print(len(embds))
print(gen[0])

In [ ]:
print(embds[0].shape)
stacked_tensor = torch.cat(embds, dim=0)
data_matrix = stacked_tensor.numpy()
num_samples = data_matrix.shape[0]
num_features = data_matrix.shape[1] * data_matrix.shape[2] * data_matrix.shape[3]
data_2d = data_matrix.reshape(num_samples, num_features)
print(f"Shape Data: {data_2d.shape}")

In [ ]:
from sklearn.decomposition import PCA
noise_pca = PCA(n_components=4)
noise_principal_components = noise_pca.fit_transform(data_2d)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
gen_nums = np.array([int(g) for g in gen])
min_gen = gen_nums.min() 
max_gen = gen_nums.max() 

plt.figure(figsize=(8, 6))

plt.scatter(
    noise_principal_components[:, 0],
    noise_principal_components[:, 1],
    c=gen_nums,
    cmap='viridis',
    s=5,
    alpha=0.7)
plt.title('2D-PCA-Plot (PC1 vs. PC2)')
plt.xlabel('PC1')
plt.ylabel('PC2')
plt.grid(True)
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
gen_nums = np.array([int(g) for g in gen])
min_gen = gen_nums.min() # Sollte 0 sein
max_gen = gen_nums.max() # Sollte 40 sein

plt.figure(figsize=(8, 6))

plt.scatter(
    noise_principal_components[:, 2],
    noise_principal_components[:, 3],
    c=gen_nums,
    cmap='viridis',
    s=5,
    alpha=0.7)
plt.title('2D-PCA-Plot (PC3 vs. PC4)')
plt.xlabel('PC3')
plt.ylabel('PC4')
plt.grid(True)
plt.show()

In [ ]:
explained_variance = noise_pca.explained_variance_ratio_
print("Explained Variance: (PC1 - PC4):")
for i in range(4):
    variance_percentage = explained_variance[i] * 100
    print(f"Component {i+1} (PC{i+1}): {variance_percentage:.2f}%")

In [ ]:
import umap

reducer = umap.UMAP()

In [ ]:
from sklearn.preprocessing import StandardScaler
n_scaler = StandardScaler()
n_scaled_data = n_scaler.fit_transform(data_2d)

In [ ]:
embedding = reducer.fit_transform(n_scaled_data)
embedding.shape

In [ ]:
plt.scatter(
    embedding[:, 0],
    embedding[:, 1],
    c=gen_nums,
    s=5,
    alpha=0.7)
plt.gca().set_aspect('equal', 'datalim')
plt.title('UMAP projection of Initial Noise', fontsize=12);

In [ ]:
pc1 = noise_pca.components_[0]
pc2 = noise_pca.components_[1]

In [ ]:
import PIL
def latents_to_rgb(latents):
    """
    Convert the SDXL latents (4 channels: Luminance, Cyan, Red, Pattern Structure) to RGB tensors (3 channels).
    Explanation: https://huggingface.co/blog/TimothyAlexisVass/explaining-the-sdxl-latent-space#the-4-channels-of-the-sdxl-latents
    """
    weights = (
        (60, -60, 25, -70),
        (60,  -5, 15, -50),
        (60,  10, -5, -35),
    )

    weights_tensor = torch.t(torch.tensor(weights, dtype=latents.dtype).to(latents.device))
    biases_tensor = torch.tensor((150, 140, 130), dtype=latents.dtype).to(latents.device)
    rgb_tensor = torch.einsum("...lxy,lr -> ...rxy", latents, weights_tensor) + biases_tensor.unsqueeze(-1).unsqueeze(-1)
    
    # Handle batch dimension - squeeze if batch size is 1
    if rgb_tensor.dim() == 4 and rgb_tensor.shape[0] == 1:
        rgb_tensor = rgb_tensor.squeeze(0)  # Remove batch dimension: [1, 3, H, W] -> [3, H, W]
    
    # Ensure we have [C, H, W] format before transpose to [H, W, C]
    if rgb_tensor.dim() == 3:
        image_array = rgb_tensor.clamp(0, 255).byte().cpu().numpy().transpose(1, 2, 0)
    else:
        raise ValueError(f"Unexpected tensor shape: {rgb_tensor.shape}. Expected [C, H, W] or [1, C, H, W]")

    return PIL.Image.fromarray(image_array)



In [ ]:
t_pc1 = torch.from_numpy(pc1)
pc_1_shaped = t_pc1.reshape(embds[0].shape)
print(embds[0].shape)
print(pc_1_shaped.shape)
t_pc2 = torch.from_numpy(pc2)
pc_2_shaped = t_pc2.reshape(embds[0].shape)
print(pc_2_shaped.shape)

In [ ]:
normal = latents_to_rgb(embds[0])
pc1_rgb =latents_to_rgb(pc_1_shaped)
pc2_rgb =latents_to_rgb(pc_2_shaped)
delta_tensor = pc_1_shaped - pc_2_shaped
delta_rgb = latents_to_rgb(delta_tensor)

In [ ]:
fig, achsen = plt.subplots(1, 4, figsize=(10, 5))

achsen[0].imshow(pc1_rgb)
achsen[0].set_title('PC1')
achsen[0].axis('off') 


achsen[1].imshow(pc2_rgb)
achsen[1].set_title('PC2')
achsen[1].axis('off') 

achsen[2].imshow(delta_rgb)
achsen[2].set_title('Delta')
achsen[2].axis('off') 


achsen[3].imshow(delta_rgb)
achsen[3].set_title('Delta')
achsen[3].axis('off') 

plt.tight_layout()


plt.show()